<a href="https://colab.research.google.com/github/Sarthakpunj/RAG_from_scratch/blob/main/rag_implementation_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
corpus = [
    "In a quiet forest, there lived a wise old owl.",
    "He rarely spoke, but he saw and heard everything from his perch high in the trees.",
    "Every day, the other animals would chatter loudly and argue with one another.",
    "The squirrel would complain about the noisy woodpecker, the fox would boast about his clever tricks, and the rabbit would gossip about everyone else.",
    "The owl listened but said nothing.",
    "One day, the animals asked the owl, \"You’re so quiet. Don’t you have anything to say?\"",
    "The owl finally replied, \"I’ve learned that the more I listen, the more I understand. And the more I talk, the less I learn.\"",
    "The forest grew quiet.",
    "From that day forward, the animals argued less and listened more.",
    "Moral: Wisdom grows when we learn to listen more and speak less."
]


In [2]:
#cosine similarity formula: A.B/|A||B|
user_query = "I am an Indian and i live in India"
Document ="India is a country of Indians and for everyone"
#finding similarity between q and doc
#step 1: convert data to vectors
from collections import Counter
import math
user_query_tokens = user_query.lower().split(" ") #lowering the capital and splitting based on space I am = {"i" and "am"}
Document_tokens = Document.lower().split(" ")
print(user_query_tokens)
print(Document_tokens)

['i', 'am', 'an', 'indian', 'and', 'i', 'live', 'in', 'india']
['india', 'is', 'a', 'country', 'of', 'indians', 'and', 'for', 'everyone']


In [13]:
# 2 making tokens to numbers , we wil use counter to count and based on frequency we assign number: this will be embedding
query_counter=Counter(user_query_tokens)
documnet_counter=Counter(Document_tokens)
print(query_counter)
print(documnet_counter)

Counter({'i': 2, 'am': 1, 'an': 1, 'indian': 1, 'and': 1, 'live': 1, 'in': 1, 'india': 1})
Counter({'india': 1, 'is': 1, 'a': 1, 'country': 1, 'of': 1, 'indians': 1, 'and': 1, 'for': 1, 'everyone': 1})


In [14]:
#3 finding cosine similarity
for tokens in query_counter.keys() & documnet_counter.keys():
    print(tokens)# this gives word common in both

india
and


In [16]:
lst = []
for tokens in query_counter.keys() & documnet_counter.keys():
    lst.append(query_counter[tokens]*documnet_counter[tokens])# perform dot product= india:1 and:1 = [1,1] (india *india = 1*1 = 1 similar with and)=[1,1] now after this we add 1 and 1 = 2
print(lst)

[1, 1]


In [20]:
dot_prod=sum(lst) # A.B

In [18]:
#magnitude of vector sqrt(a**2 +b**@)
query_mag = math.sqrt(sum(query_counter[tokens]**2 for tokens in query_counter))
documnet_mag = math.sqrt(sum(documnet_counter[tokens]**2 for tokens in documnet_counter))
print(query_mag)
print(documnet_mag)

3.3166247903554
3.0


In [21]:
similarity = (dot_prod)/(query_mag*documnet_mag)
print(similarity)

0.20100756305184242


In [22]:
#creating a function
def cosine_similarity(query, document):
    # Tokenize and convert to lowercase
    query_tokens = query.lower().split(" ")
    document_tokens = document.lower().split(" ")

    # Create Counters for query and document
    query_counter = Counter(query_tokens)
    document_counter = Counter(document_tokens)

    # Calculate dot product
    dot_product = sum(query_counter[token] * document_counter[token] for token in query_counter.keys() & document_counter.keys())

    # Calculate magnitudes
    query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))
    document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))

    # Calculate cosine similarity
    similarity = dot_product / (query_magnitude * document_magnitude) if query_magnitude * document_magnitude != 0 else 0

    return similarity

In [23]:
user_query = "I am an Indian and i live in India"
Document ="India is a country of Indians and for everyone"
cosine_similarity(user_query, Document)

0.20100756305184242

In [27]:
#this method will take user query and whol corpus for each document in corpus it will find similarity of each document with user query and we will get a score then we find which is the max score/ most relatable doc from corpus and returen that
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = cosine_similarity(query, doc)
        similarities.append(similarity)
    return corpus[similarities.index(max(similarities))]

In [26]:
corpus

['In a quiet forest, there lived a wise old owl.',
 'He rarely spoke, but he saw and heard everything from his perch high in the trees.',
 'Every day, the other animals would chatter loudly and argue with one another.',
 'The squirrel would complain about the noisy woodpecker, the fox would boast about his clever tricks, and the rabbit would gossip about everyone else.',
 'The owl listened but said nothing.',
 'One day, the animals asked the owl, "You’re so quiet. Don’t you have anything to say?"',
 'The owl finally replied, "I’ve learned that the more I listen, the more I understand. And the more I talk, the less I learn."',
 'The forest grew quiet.',
 'From that day forward, the animals argued less and listened more.',
 'Moral: Wisdom grows when we learn to listen more and speak less.']

In [38]:
user_query = "what did owl listen?"
return_response(user_query, corpus)

'The owl listened but said nothing.'

In [45]:
#rag is made now we configure llm to generate repsonse through gemini

from google.colab import userdata
import google.generativeai as genai
#api= userdata.get('gemini')
#configuring
genai.configure(api_key=api)

model = genai.GenerativeModel('gemini-2.0-flash')

user_input = "What lesson did the animals learn from the owl?"
relevant_document = return_response(user_input, corpus)

prompt = f"""
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""




In [46]:
#response = model.generate_content(prompt)
#print(response.text)